In [2]:
import os

### How Javelin and CrewwAI Integrate

## Javelin Setup:

- **API Keys**: Javelin provides an API key (`x-api-key`) to authenticate the requests.
- **Javelin Route**: Defines the specific route to a task or flow.

## Calling CrewwAI via Javelin:

- **CrewwAI** utilizes Javelin’s routes to send and manage requests.
  
  - **Flow Definition**: CrewwAI uses the Javelin API to define agents (e.g., market analyst, strategist) and their corresponding tasks.
  
  - **Task Execution**: Each agent, when triggered via the route, will execute the specific task as per the flow defined in CrewwAI.
  
  - **Flow Management**: CrewwAI listens to each agent’s output and triggers the next step or task, ensuring smooth workflow orchestration.


### Flow Execution Order:

##### The tasks are now executed in sequence: research_task → project_understanding_task → marketing_strategy_task → campaign_idea_task → copy_creation_task

No Task Skipped:

All tasks are connected in a linear chain. The flow will not skip any task unless an error occurs or a condition is met that stops execution.
Using @listen() Decorators:

Each task is marked with @listen(), which means they will listen for the completion of the preceding task and then take the result as input for the next task.
Result:

The flow will execute all tasks, starting from the market research all the way to copy creation.
The final output, Final Marketing Copies, will be printed when the flow completes.

In [32]:
import yaml
import os
import asyncio
from dotenv import load_dotenv
from openai import OpenAI, AsyncOpenAI
from crewai.flow.flow import Flow, listen, start
from javelin_sdk import JavelinClient, JavelinConfig

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
javelin_api_key = os.getenv("JAVELIN_API_KEY")

if not javelin_api_key or not openai_api_key:
    raise ValueError("API keys not found. Ensure .env file is properly loaded.")

# Initialize Javelin Client
config = JavelinConfig(javelin_api_key=javelin_api_key)
javelin_client = JavelinClient(config)
openai_client = OpenAI(api_key=openai_api_key)
javelin_client.register_openai(openai_client, route_name="openai_univ")

# Load YAML Configurations
def load_yaml(file_path):
    assert os.path.exists(file_path), f"{file_path} not found!"
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

agents_config = load_yaml("agents/agents.yaml")
tasks_config = load_yaml("agents/tasks.yaml")

class MarketingFlow(Flow):
    model = "gpt-3.5-turbo"

    @start()
    def research_task(self):
        """Conducts market research."""
        task = tasks_config["research_task"]
        prompt = task["description"].format(customer_domain="Tech Industry", project_description="AI-Powered CRM")

        response = openai_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        research_report = response.choices[0].message.content
        self.state["research_report"] = research_report
        return research_report

    @listen(research_task)
    def project_understanding_task(self, research_report):
        """Understands the project details."""
        task = tasks_config["project_understanding_task"]
        prompt = task["description"].format(project_description="AI-Powered CRM")

        response = openai_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        project_summary = response.choices[0].message.content
        self.state["project_summary"] = project_summary
        return project_summary

    @listen(project_understanding_task)
    def marketing_strategy_task(self, project_summary):
        """Develops marketing strategies."""
        task = tasks_config["marketing_strategy_task"]
        prompt = task["description"].format(customer_domain="Tech Industry", project_description="AI-Powered CRM")

        response = openai_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        strategy_doc = response.choices[0].message.content
        self.state["marketing_strategy"] = strategy_doc
        return strategy_doc

    @listen(marketing_strategy_task)
    def campaign_idea_task(self, marketing_strategy):
        """Creates marketing campaign ideas."""
        task = tasks_config["campaign_idea_task"]
        prompt = task["description"].format(project_description="AI-Powered CRM")

        response = openai_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        campaign_ideas = response.choices[0].message.content
        self.state["campaign_ideas"] = campaign_ideas
        return campaign_ideas

    @listen(campaign_idea_task)
    def copy_creation_task(self, campaign_ideas):
        """Writes marketing copies based on campaign ideas."""
        task = tasks_config["copy_creation_task"]
        prompt = task["description"].format(project_description="AI-Powered CRM")

        response = openai_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        marketing_copies = response.choices[0].message.content
        self.state["marketing_copies"] = marketing_copies
        return marketing_copies

# Run Flow Asynchronously
async def run_flow():
    flow = MarketingFlow()
    result = await flow.kickoff_async()
    print(f"Final Marketing Copies: {result}")

if __name__ == "__main__":
    asyncio.run(run_flow())


Final Marketing Copies: 1. "Unleash Your Potential with Our Fitness Challenge! Join our 30-day challenge to push your limits, reach your goals, and transform your body. Are you ready to sweat, work hard, and see real results? Sign up today and let's crush those fitness goals together!"

2. "Upgrade Your Skincare Routine with Our New Beauty Box! Say goodbye to dull, tired skin and hello to a radiant, glowing complexion. Our curated beauty box is filled with top-quality products to nourish and hydrate your skin. Treat yourself to the ultimate self-care experience and embrace your natural beauty today!"

3. "Fuel Your Adventures with Our Healthy Snack Pack! Whether you're hitting the trails, heading to the gym, or just need a quick pick-me-up, our snack pack has got you covered. Packed with protein-rich snacks and delicious treats, you'll have the fuel you need to conquer any challenge. Grab a pack today and stay energized on-the-go!"

4. "Revamp Your Home Decor with Our Stylish Furniture